In [1]:
from pyspark import *
from pyspark import SparkConf, SparkContext
from pyspark.sql import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *

conf = SparkConf().setMaster("local[*]").setAppName("test")
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")

spark = SparkSession.builder.getOrCreate()

In [2]:
data1 = [
    (1, "A", "A", 1000000),
    (2, "B", "A", 2500000),
    (3, "C", "G", 500000),
    (4, "D", "G", 800000),
    (5, "E", "W", 9000000),
    (6, "F", "W", 2000000),
]
df1 = spark.createDataFrame(data1, ["emp_id", "name", "dept_id", "salary"])
df1.show()

data2 = [("A", "AZURE"), ("G", "GCP"), ("W", "AWS")]
df2 = spark.createDataFrame(data2, ["dept_id1", "dept_name"])
df2.show()

d:\bigdata\pyspark\python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


+------+----+-------+-------+
|emp_id|name|dept_id| salary|
+------+----+-------+-------+
|     1|   A|      A|1000000|
|     2|   B|      A|2500000|
|     3|   C|      G| 500000|
|     4|   D|      G| 800000|
|     5|   E|      W|9000000|
|     6|   F|      W|2000000|
+------+----+-------+-------+

+--------+---------+
|dept_id1|dept_name|
+--------+---------+
|       A|    AZURE|
|       G|      GCP|
|       W|      AWS|
+--------+---------+



In [3]:
joindf = df1.join(df2, df1["dept_id"] == df2["dept_id1"], "inner").drop("dept_id1")
joindf.show()

+------+----+-------+-------+---------+
|emp_id|name|dept_id| salary|dept_name|
+------+----+-------+-------+---------+
|     1|   A|      A|1000000|    AZURE|
|     2|   B|      A|2500000|    AZURE|
|     5|   E|      W|9000000|      AWS|
|     6|   F|      W|2000000|      AWS|
|     3|   C|      G| 500000|      GCP|
|     4|   D|      G| 800000|      GCP|
+------+----+-------+-------+---------+



In [5]:
wn = Window.partitionBy("dept_id").orderBy(col("salary").desc())

rankdf = joindf.withColumn("rank", dense_rank().over(wn))
rankdf.show()

+------+----+-------+-------+---------+----+
|emp_id|name|dept_id| salary|dept_name|rank|
+------+----+-------+-------+---------+----+
|     2|   B|      A|2500000|    AZURE|   1|
|     1|   A|      A|1000000|    AZURE|   2|
|     5|   E|      W|9000000|      AWS|   1|
|     6|   F|      W|2000000|      AWS|   2|
|     4|   D|      G| 800000|      GCP|   1|
|     3|   C|      G| 500000|      GCP|   2|
+------+----+-------+-------+---------+----+



In [6]:
finaldf = (
    rankdf.filter(col("rank") == 2)
    .drop("rank")
    .select("emp_id", "name", "dept_name", "salary")
)
finaldf.show()

+------+----+---------+-------+
|emp_id|name|dept_name| salary|
+------+----+---------+-------+
|     1|   A|    AZURE|1000000|
|     6|   F|      AWS|2000000|
|     3|   C|      GCP| 500000|
+------+----+---------+-------+

